In [1]:
from mlx_lm import load, generate

/Users/miaostar/miniforge3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time
model, tokenizer = load("mlx-community/Meta-Llama-3-8B-Instruct-4bit")

Fetching 6 files: 100%|████████████████████████| 6/6 [00:00<00:00, 76959.71it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 626 ms, sys: 1.38 s, total: 2.01 s
Wall time: 3.09 s


### The default example of code generation
[link](https://huggingface.co/mlx-community/Meta-Llama-3-8B-Instruct-4bit#use-with-mlx)

In [3]:
response = generate(model, tokenizer, prompt="You are a friendly chatbot. hello, what's your name?", verbose=True)

Prompt: You are a friendly chatbot. hello, what's your name?
")

    # Check if the user's message is in the list of expected messages
    if message in expected_messages:
        # If the user's message is in the list of expected messages, respond with a friendly message
        response = "Hello! My name is Chatbot. I'm here to help you with any questions or concerns you may have. How can I assist you today?"
    else:
        # If the user's message is not in the list of expected messages, respond with
Prompt: 49.385 tokens-per-sec
Generation: 19.349 tokens-per-sec


The response is more like a continuation of some pseudo code. Not what we expected. To fix it, let's apply the [chat template](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct#transformers-automodelforcausallm)!

### Apply the chat template

In [4]:
messages = [
    {"role": "system", "content": "You are a friendly chatbot."},
    {"role": "user", "content": "hello, what's your name?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
)
input_ids

[128000,
 128006,
 9125,
 128007,
 271,
 2675,
 527,
 264,
 11919,
 6369,
 6465,
 13,
 128009,
 128006,
 882,
 128007,
 271,
 15339,
 11,
 1148,
 596,
 701,
 836,
 30,
 128009,
 128006,
 78191,
 128007,
 271]

input_ids is a list of integers but the `generate` function expects input prompt to be string.

So we convert it back to a string by `tokenizer.decode`

In [5]:
prompt = tokenizer.decode(input_ids)
prompt

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a friendly chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nhello, what's your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In this new prompt, we can clearly see templates applied to our original question.

In [6]:
response = generate(model, tokenizer, prompt=prompt, verbose=True)

Prompt: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a friendly chatbot.<|eot_id|><|start_header_id|>user<|end_header_id|>

hello, what's your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Hello! My name is Chatty, and I'm a friendly chatbot. I'm here to help answer your questions, provide information, and have a nice conversation with you. How are you today?
Prompt: 115.986 tokens-per-sec
Generation: 19.447 tokens-per-sec


Now the response looks much better!